In [12]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from dateutil import parser
from datetime import datetime



#initiate data storage
Date = []
Author = []
Title = []
Articles = []
PDFlink = []
df = pd.DataFrame(columns=["Date","Author","Title","Articles"])
href='<a href="publications/news/774001_22092020-1.pdf"'
start = href.find('<a href="')
for page in range(1,5):
        r = requests.get("http://www.bnacapitaux.com.tn/?pg="+str(page)+"&news=all")
        soup =   BeautifulSoup(r.content,"lxml")
        #Extract date
        for dateDiv in soup.find_all(class_="NewsDate")[:6]:
            match = re.search(r'\d{2}/\d{2}/\d{4}', dateDiv.text.replace("\n",""))
            datetime = parser.parse(match.group())
            Date.append(datetime)
        
        #Extract author
        for author in soup.find_all(class_="NewsDate")[:6]:
            
                Author.append(author.text.replace("\n","").strip()[25:-1])
            
                
        #Extract title       
        for title in soup.find_all(class_="NewsTitle")[:6]:
            Title.append(title.text.replace("\n","").strip())
            
        #Extract description    
        for article in soup.find_all('div', id=lambda x: x and x.startswith('NEW_DESC'))[:6]:
            
                Articles.append(article.text.replace("\n","").strip())
                
        #Extract PDFlink        
        liste=[]
        liste1=[]
        for link in soup.find_all('a', href=lambda x: x and x.startswith('publications')):
            match = re.search(r'\bAttachement\b', str(link))
            if match !=None :
                liste.append(str(link)[9:60])
                for l in liste:
                    pos= l.find('"')
                    l=("http://www.bnacapitaux.com.tn/"+l[0:pos])
                liste1.append(l)     
        PDFlink=PDFlink+liste1[0:6]

#Fill the dataframe
for i in range(len(Date)):
    df = df.append({'Date':Date[i],'Author':Author[i],'Title':Title[i],'Articles':Articles[i],'PDF':PDFlink[i]},ignore_index=True)

In [14]:
df['PDF'][5]

'http://www.bnacapitaux.com.tn/publications/news/763001_20102020-1.pdf'

In [33]:
df.to_excel(r'./ArticlesBNAcapitaux.xlsx', index = False)

In [2]:
!pip install bs4

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Using cached beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)
  Using cached soupsieve-2.0.1-py3-none-any.whl (32 kB)
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=67436ac005e0a6615ffde1968eddfa54ea5224f5feebaeb176d83e23ab657761
  Stored in directory: c:\users\amin\appdata\local\pip\cache\wheels\0a\9e\ba\20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
